In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Sp/Imagenes_confocal/sp_para entrenar/'
pathlist = Path(path).glob('*.csv')
sparks_data_list = []
for file in pathlist:
    path_in_str = str(file)
    df = pd.read_csv(path_in_str)
    sparks_data_list.append(df)
sparks_data_list = pd.concat(sparks_data_list)
del (sparks_data_list['Unnamed: 0'])
del (sparks_data_list['minim'])
del (sparks_data_list['FWHM'])
del (sparks_data_list['pos_x'])
del (sparks_data_list['pos_y'])
del (sparks_data_list['FDHM'])
del (sparks_data_list['full_width'])
del (sparks_data_list['sparks_tiempo_pico50_2'])
del (sparks_data_list['tiempo_minimo'])

In [ ]:
sparks_data_list.to_csv(path + 'sparks_data_list.csv')

In [ ]:
#Disabling the warnings permantently
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier


from sklearn.model_selection import (RandomizedSearchCV,
                                     KFold,
                                     cross_val_score,
                                     train_test_split)
from sklearn.metrics import (roc_auc_score,
                             make_scorer,
                             classification_report,
                             roc_curve,
                             f1_score,
                             matthews_corrcoef,
                             precision_score,
                             recall_score,
                             confusion_matrix)

from xgboost import XGBClassifier

In [ ]:
sparks_data_list.isnull().sum()

In [ ]:
corr = sparks_data_list[sparks_data_list.columns].corr()
sns.heatmap(corr, cmap="YlGnBu", annot = True)
sns.set(rc={'figure.figsize':(30,24)})

In [ ]:
X = sparks_data_list.drop(['flag'], axis=1).values
y = sparks_data_list.flag

In [ ]:
X_train,X_test,  y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    stratify=y)

In [ ]:
df_train = pd.DataFrame(y_train)
df_test = pd.DataFrame(y_test)

In [ ]:
print(df_train.groupby('flag').size())
print(df_test.groupby('flag').size())

In [ ]:
from sklearn.impute import SimpleImputer

# Tratamiento de NaN
imp = SimpleImputer(missing_values=np.nan, strategy='median') # Uso el promedio para reemplazar
imp.fit(X_train)
X_train = imp.transform(X_train)

imp.fit(X_test)
X_test = imp.transform(X_test)

In [ ]:
model_baseline = XGBClassifier()

In [ ]:
model_baseline.fit(X_train, y_train)

In [ ]:
def get_prediction(predictions):
    new_preds = []
    for preds in predictions:
        if preds >= 0.25:
            new_preds.append(1)
        else: new_preds.append(2)
    return new_preds

In [ ]:
pred_proba_train = model_baseline.predict_proba(X_train)[:, 1]
pred_proba_test = model_baseline.predict_proba(X_test)[:, 1]

pred_train = get_prediction(pred_proba_train)
pred_test = get_prediction(pred_proba_test)

In [ ]:
print(f"Train ROC-AUC: {np.round(roc_auc_score(y_train, pred_proba_train), 2)}")
print(f"Test ROC-AUC: {np.round(roc_auc_score(y_test, pred_proba_test), 2)}")

In [ ]:
import joblib

# Save the model as a pickle in a file
# joblib.dump(model_baseline, 'modelXGB.pkl')

# Load the model from the file
knn_from_joblib = joblib.load('modelXGB.pkl')

# Use the loaded model to make predictions
# knn_from_joblib.predict(X_test)


pred_proba_train = model_baseline.predict_proba(X_train)[:, 1]
pred_proba_test = knn_from_joblib.predict_proba(X_test)[:, 1]

pred_train = get_prediction(pred_proba_train)
pred_test = get_prediction(pred_proba_test)

print(f"Train ROC-AUC: {np.round(roc_auc_score(y_train, pred_proba_train), 2)}")
print(f"Test ROC-AUC: {np.round(roc_auc_score(y_test, pred_proba_test), 2)}")

In [ ]:
import Image_Sparks
import cv2
import sparks_analysis_apply
import image_class

# path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Sp/Imagenes_confocal/sp_para entrenar/'
image = image_class.Image_Processing('/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Sp/Imagenes_confocal/Rata/C071112/', 'c1ac000')


image = cv2.imread(image.path + image.photo_name + ".tif")
imcrop = Image_Sparks.Image_Sparks.Image_cropping(image)
cv2.imwrite('/path/to/destination/image.png',imcrop)
contours, img_processed = Image_Sparks.Image_Sparks.Image_processing(imcrop)
list_img_col, list_img_row, x_position, y_position, width, high = Image_Sparks.Image_Sparks.Image_analysis(contours, img_processed, image)

# Take the processing data to analyze each spark and extract parameters

parameters = sparks_analysis_apply.analysis_process (list_img_col, list_img_row,x_position,y_position,width,high)
parameters = parameters.drop(['FWHM','FDHM','pos_x','pos_y','tau','width','high'], axis = 1)
# knn_from_joblib.predict(parameters).to_numpy()
print(parameters)